In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
import tensorflow_probability as tfp
tfk = tf.keras
tf.keras.backend.set_floatx('float64')
tfd = tfp.distributions
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-08-03 11:47:58.845162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 11:47:59.271212: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 11:47:59.363301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-03 11:47:59.363397: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

In [3]:
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

#### The data

In [4]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [5]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [6]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [7]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [8]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### Model training and validation

In [9]:
def bnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tfp.layers.DenseVariational(units=128, make_prior_fn=tfp.layers.default_multivariate_normal_fn, 
#                                 make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=64, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=1, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     ])
    
# #     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())
#     model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
#     model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(1, dtype=tf.float64), scale=1.0), 
                            reinterpreted_batch_ndims=1)
    model = tfk.Sequential([
        tfk.layers.InputLayer(input_shape=(3,), name='input'),
        tfk.layers.Dropout(dropout, name='dropout1'),
        tfk.layers.Dense(10, activation='relu', name='dense_1'),
        tfk.layers.Dropout(dropout, name='dropout2'),
        tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(1), activation=None, name='distribution_weights'),
        tfk.layers.Dropout(dropout, name='dropout3'),
        tfp.layers.MultivariateNormalTriL(1, activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/32), 
                                          name='output')], name='model')
    
#     model.compile(optimizer=optimizer, loss=neg_log_likelihood)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    return model

In [10]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
#     data_train =tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
#     data_train = data_train.batch(32).repeat(n_epochs)
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
#     data_test =tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
#     data_test = data_test.batch(1)
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
    
    model = bnn(X_train, y_train) #dropout=0.2
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [12]:
# rmse_list, mape_list = [], []
# for i in range(len(latitudes)):
#     rmse, mape = cross_validation(final_df, i)
#     rmse_list.append(rmse)
#     mape_list.append(mape)
#     print(f'{device_ids[i]} successful')

In [13]:
rmse_list, mape_list = [], []
# for i in range(len(latitudes[30:])):
lat_list = list(latitudes)
for lat in latitudes[:10]:
    i = lat_list.index(lat)
#     print(i)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

2023-08-03 11:48:53.744796: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-03 11:48:53.745046: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-08-03 11:48:53.745106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-08-03 11:48:53.745614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-08-03 11:48:53.745822: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.125.06  Release Build  (dvs-builder@U16-A23-14-1)  Tue May 30 05:12:48 UTC 2023
GCC version:  gcc ve

Epoch 1/1000
1155/1155 [==============================] - 8s 5ms/step - loss: 3456352749.0508 - val_loss: 890.8554
Epoch 2/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 1031.6575 - val_loss: 694.9715
Epoch 3/1000
1155/1155 [==============================] - 4s 4ms/step - loss: 976.0320 - val_loss: 1065.6290
Epoch 4/1000
1155/1155 [==============================] - 4s 3ms/step - loss: 963.8915 - val_loss: 1012.1180
Epoch 5/1000
1155/1155 [==============================] - 4s 3ms/step - loss: 951.3565 - val_loss: 792.9047
Epoch 6/1000
1155/1155 [==============================] - 4s 4ms/step - loss: 935.9941 - val_loss: 891.9057
Epoch 7/1000
1155/1155 [==============================] - 4s 4ms/step - loss: 920.6640 - val_loss: 941.5761
Epoch 8/1000
1155/1155 [==============================] - 5s 4ms/step - loss: 919.1619 - val_loss: 753.1249
Epoch 9/1000
1155/1155 [==============================] - 4s 4ms/step - loss: 911.9843 - val_loss: 861.1488
Epoch 10/1000
1155

1155/1155 [==============================] - 6s 5ms/step - loss: 785.9265 - val_loss: 683.3072
Epoch 77/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 788.7490 - val_loss: 683.1657
Epoch 78/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 792.7752 - val_loss: 683.7197
Epoch 79/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 792.3787 - val_loss: 683.7659
Epoch 80/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 792.3377 - val_loss: 684.0611
Epoch 81/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 788.1905 - val_loss: 683.9052
Epoch 82/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 789.5681 - val_loss: 683.8209
Epoch 83/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 784.1833 - val_loss: 683.9195
Epoch 84/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 784.8812 - val_loss: 683.0602
Epoch 85/1000
1155/1155 [========

Epoch 151/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 785.2408 - val_loss: 683.8870
Epoch 152/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 802.8627 - val_loss: 684.0547
Epoch 153/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 788.7613 - val_loss: 684.0405
Epoch 154/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 784.2238 - val_loss: 683.2921
Epoch 155/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 791.2985 - val_loss: 683.4083
Epoch 156/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 784.0790 - val_loss: 683.2834
Epoch 157/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 787.7545 - val_loss: 682.9561
Epoch 158/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 794.6252 - val_loss: 683.5380
Epoch 159/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 790.2162 - val_loss: 683.7264
Epoch 160/

1155/1155 [==============================] - 5s 4ms/step - loss: 790.1317 - val_loss: 684.0446
Epoch 226/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 783.0923 - val_loss: 683.8140
Epoch 227/1000
1155/1155 [==============================] - 5s 4ms/step - loss: 785.2978 - val_loss: 683.5023
Epoch 228/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 793.1829 - val_loss: 683.8990
Epoch 229/1000
1155/1155 [==============================] - 6s 5ms/step - loss: 792.0516 - val_loss: 684.2972
Epoch 230/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 785.3574 - val_loss: 684.0334
Epoch 231/1000
1155/1155 [==============================] - 5s 4ms/step - loss: 784.9862 - val_loss: 683.7960
Epoch 232/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 782.2519 - val_loss: 683.5740
Epoch 233/1000
1155/1155 [==============================] - 5s 5ms/step - loss: 787.8465 - val_loss: 683.9918
Epoch 234/1000
1155/1155 

...

1171/1171 [==============================] - 4s 3ms/step - loss: 782.7613 - val_loss: 678.9607
Epoch 338/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 786.2090 - val_loss: 678.5507
Epoch 339/1000
1171/1171 [==============================] - 4s 4ms/step - loss: 791.3921 - val_loss: 679.0387
Epoch 340/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 787.3267 - val_loss: 679.0698
Epoch 341/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 785.2787 - val_loss: 679.0664
Epoch 342/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 780.6068 - val_loss: 678.7559
Epoch 343/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 779.9781 - val_loss: 678.6821
Epoch 344/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 780.0591 - val_loss: 678.0929
Epoch 345/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 779.2125 - val_loss: 678.4561
Epoch 346/1000
1171/1171 

1171/1171 [==============================] - 4s 3ms/step - loss: 786.1841 - val_loss: 678.4312
Epoch 412/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 787.2482 - val_loss: 678.7824
Epoch 413/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 781.6036 - val_loss: 678.5503
Epoch 414/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 779.6931 - val_loss: 678.6114
Epoch 415/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 789.2968 - val_loss: 678.8770
Epoch 416/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 784.0634 - val_loss: 679.2071
Epoch 417/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 784.5475 - val_loss: 678.9413
Epoch 418/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 794.6200 - val_loss: 679.2867
Epoch 419/1000
1171/1171 [==============================] - 4s 4ms/step - loss: 785.0059 - val_loss: 679.2491
Epoch 420/1000
1171/1171 

1171/1171 [==============================] - 4s 3ms/step - loss: 794.7157 - val_loss: 679.0269
Epoch 486/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 782.3014 - val_loss: 679.1233
Epoch 487/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 785.1371 - val_loss: 678.8993
Epoch 488/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 781.9483 - val_loss: 678.6965
Epoch 489/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 780.9269 - val_loss: 678.5680
Epoch 490/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 777.5369 - val_loss: 678.1851
Epoch 491/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 788.9433 - val_loss: 678.4742
Epoch 492/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 785.2454 - val_loss: 678.4066
Epoch 493/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 786.5494 - val_loss: 678.6837
Epoch 494/1000
1171/1171 

1171/1171 [==============================] - 4s 3ms/step - loss: 785.1550 - val_loss: 678.6765
Epoch 560/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 788.2736 - val_loss: 678.6236
Epoch 561/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 788.3421 - val_loss: 678.6616
Epoch 562/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 787.2499 - val_loss: 678.3665
Epoch 563/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 781.3482 - val_loss: 678.0593
Epoch 564/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 783.0005 - val_loss: 677.7351
Epoch 565/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 781.3008 - val_loss: 678.0321
Epoch 566/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 786.6488 - val_loss: 677.9872
Epoch 567/1000
1171/1171 [==============================] - 4s 3ms/step - loss: 786.0691 - val_loss: 677.8185
Epoch 568/1000
1171/1171 

KeyboardInterrupt: 

In [14]:
rmse_list

[16.538257089884546, 16.84502278200903, 10.579277373251326]

In [ ]:
mape_list

In [ ]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

In [ ]:
mean_mape = np.mean(mape_list)
mean_mape